In [2]:
import os
import numpy as np
import cv2

In [4]:
from tensorflow.keras.utils import to_categorical

In [5]:
from keras.layers import Input, Dense 
from keras.models import Model

In [6]:
is_init = False
size = -1

label = []
dictionary = {}
c = 0

In [7]:
for file_name in os.listdir():
    if file_name.endswith(".npy") and not file_name.startswith("labels"):
        data = np.load(file_name)
        num_samples = data.shape[0]

        if num_samples > 0:
            if not is_init:
                is_init = True
                X = data if data.ndim > 1 else data.reshape(-1, 1)
                y = np.array([file_name.split('.')[0]] * num_samples).reshape(-1, 1)
            else:
                if data.ndim == 1:
                    data = data.reshape(-1, 1)
                
                if data.shape[1] != X.shape[1]:
                    print(f"Skipping file {file_name} due to shape mismatch: {data.shape} vs {X.shape}")
                    continue

                X = np.concatenate((X, data))
                y_labels = np.array([file_name.split('.')[0]] * num_samples).reshape(-1, 1)
                y = np.concatenate((y, y_labels))

            label.append(file_name.split('.')[0])
            dictionary[file_name.split('.')[0]] = c
            c += 1
        else:
            print(f"Empty data array for file: {file_name}")

# After processing all files, convert label to a numpy array
label = np.array(label)

Skipping file emotion.npy due to shape mismatch: (1, 1) vs (200, 1020)


In [8]:
for i in range(y.shape[0]):
	y[i, 0] = dictionary[y[i, 0]]
y = np.array(y, dtype="int32")

In [9]:
y = to_categorical(y)

In [10]:
X_new = X.copy()
y_new = y.copy()
counter = 0 

cnt = np.arange(X.shape[0])
np.random.shuffle(cnt)

In [11]:
for i in cnt: 
	X_new[counter] = X[i]
	y_new[counter] = y[i]
	counter = counter + 1

In [20]:
print("Shape of X:", X.shape)

Shape of X: (400, 1020)


In [22]:
ip = Input(shape=(X.shape[1],))

In [24]:
ip = Input(shape=(X.shape[1],))
m = Dense(512, activation="relu")(ip)
m = Dense(256, activation="relu")(m)
op = Dense(y.shape[1], activation="softmax")(m)

model = Model(inputs=ip, outputs=op)
model.compile(optimizer='rmsprop', loss="categorical_crossentropy", metrics=['acc'])
model.fit(X, y, epochs=50)

Epoch 1/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - acc: 0.3589 - loss: 1.3550
Epoch 2/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - acc: 0.4731 - loss: 1.0706
Epoch 3/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - acc: 0.6003 - loss: 0.9465
Epoch 4/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - acc: 0.4309 - loss: 1.1789
Epoch 5/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - acc: 0.5241 - loss: 0.9083
Epoch 6/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - acc: 0.5325 - loss: 0.9637
Epoch 7/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - acc: 0.6419 - loss: 0.8050
Epoch 8/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - acc: 0.6881 - loss: 0.7810
Epoch 9/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - acc: 0.7753 - loss: 0.6894
Epoch 10/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - acc: 0.6995 - loss: 0.6694
Epoch 11/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - acc: 0.7669 - loss: 0.5638
Epoch 12/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - acc: 0.8670 - loss: 0.4897
Epoch 13/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2

In [26]:
model.save("my_model.keras")
np.save("labels.npy", np.array(label))